In [2]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
# from core.dl_training.scripts.training import main

In [3]:
%%bash 
pwd

/content/Template_Project


In [61]:
import toml
config_file = toml.load("configs/default_train_config.toml")
# from core.dl_framework.utils import read_config
# setup_config = read_config(config_file)

In [71]:
# config_file = toml.load("configs/default_train_config.toml")
# config_file
# test = {}
# for key, value in config_file.items():
#     if key == "title":
#         continue
#     else:
#         for subkey, subval in value.items():
#             test[subkey] = subval
#     break

config_file = toml.load("configs/default_train_config.toml")
    


def rec_dict_length(data):
    i = 0
    if type(data) == dict:
        for key, value in data.items():
            if type(value) == dict:
                print(key,value)
                return rec_dict_length(value)
            else:
                print(key, value)
    else: 
        print(data)
            
for key, value in config_file.items():
    rec_dict_length(value) 


Training configuration
gpu False
tmp_data_path ./tmp_files
num_epochs 100
num_classes 10
valid_split 0.2
source torchvision
set MNIST
batch_size 64
lr 1e-05
monitor {'monitor': ['valid_acc', 'valid_loss', 'loss']}
monitor ['valid_acc', 'valid_loss', 'loss']


In [72]:
config_file

{'title': 'Training configuration',
 'mode': {'gpu': False},
 'paths': {'tmp_data_path': './tmp_files'},
 'general': {'num_epochs': 100, 'num_classes': 10, 'valid_split': 0.2},
 'source': {'source': 'torchvision', 'set': 'MNIST'},
 'hyperparams': {'batch_size': 64, 'lr': 1e-05},
 'callbacks': {'monitor': {'monitor': ['valid_acc', 'valid_loss', 'loss']},
  'earlystopping': {'monitor': 'valid_loss', 'patience': 1000}}}

In [14]:
from core.dl_training.scripts.training import main

In [ ]:
main("/configs/default_train_config.toml")